<a href="https://colab.research.google.com/github/vivek-a81/EVA6/blob/main/Session12/Session_12_MNIST_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vit-pytorch

In [ ]:
import torch
import torchvision
from vit_pytorch import ViT
import time
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
torch.manual_seed(97)

Dpath = '/data/mnist'
Bs_Train = 100
Bs_Test = 1000

tform_mnist = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                  torchvision.transforms.Normalize((0.1307,),(0.3081,))])

tr_set = torchvision.datasets.MNIST(Dpath, train = True, download = True,
                                       transform = tform_mnist)

tr_load = torch.utils.data.DataLoader(tr_set, batch_size = Bs_Train, shuffle = True)

ts_set = torchvision.datasets.MNIST(Dpath, train = False, download = True, transform = tform_mnist)

ts_load = torch.utils.data.DataLoader(ts_set, batch_size = Bs_Test, shuffle = True)


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw




Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw




Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw




Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
def train_iter(model, optimz, data_load, loss_val):
    samples = len(data_load.dataset)
    model.train()
    
    for i, (data, target) in enumerate(data_load):
        optimz.zero_grad()
        out = F.log_softmax(model(data), dim=1)
        loss = F.nll_loss(out, target)
        loss.backward()
        optimz.step()
        
        if i % 100 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_load)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_val.append(loss.item())

def evaluate(model, data_load, loss_val):
    model.eval()
    
    samples = len(data_load.dataset)
    csamp = 0
    tloss = 0

    with torch.no_grad():
        for data, target in data_load:
            output = F.log_softmax(model(data), dim=1)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            tloss += loss.item()
            csamp += pred.eq(target).sum()

    aloss = tloss / samples
    loss_val.append(aloss)
    print('\nAverage test loss: ' + '{:.4f}'.format(aloss) +
          '  Accuracy:' + '{:5}'.format(csamp) + '/' +
          '{:5}'.format(samples) + ' (' +
          '{:4.2f}'.format(100.0 * csamp / samples) + '%)\n')

In [ ]:
N_EPOCHS = 25

start_time = time.time()
model = ViT(image_size=28, patch_size=4, num_classes=10, channels=1,
            dim=64, depth=6, heads=8, mlp_dim=128)
optimz = optim.Adam(model.parameters(), lr=0.003)

trloss_val, tsloss_val = [], []
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, tr_load, trloss_val)
    evaluate(model, ts_load, tsloss_val)

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/60000 (  0%)]  Loss: 2.4903
[10000/60000 ( 17%)]  Loss: 0.3927
[20000/60000 ( 33%)]  Loss: 0.2599
[30000/60000 ( 50%)]  Loss: 0.4195
[40000/60000 ( 67%)]  Loss: 0.1388
[50000/60000 ( 83%)]  Loss: 0.2387

Average test loss: 0.1929  Accuracy: 9409/10000 (94.09%)

Epoch: 2
[    0/60000 (  0%)]  Loss: 0.2702
[10000/60000 ( 17%)]  Loss: 0.1712
[20000/60000 ( 33%)]  Loss: 0.1816
[30000/60000 ( 50%)]  Loss: 0.2019
[40000/60000 ( 67%)]  Loss: 0.1039
[50000/60000 ( 83%)]  Loss: 0.1804

Average test loss: 0.1363  Accuracy: 9565/10000 (95.65%)

Epoch: 3
[    0/60000 (  0%)]  Loss: 0.1500
[10000/60000 ( 17%)]  Loss: 0.1183
[20000/60000 ( 33%)]  Loss: 0.1903
[30000/60000 ( 50%)]  Loss: 0.2042
[40000/60000 ( 67%)]  Loss: 0.0902
[50000/60000 ( 83%)]  Loss: 0.1612

Average test loss: 0.1030  Accuracy: 9674/10000 (96.74%)

Epoch: 4
[    0/60000 (  0%)]  Loss: 0.0119
[10000/60000 ( 17%)]  Loss: 0.0712
[20000/60000 ( 33%)]  Loss: 0.0815
[30000/60000 ( 50%)]  Loss: 0.0431
[40000/60000 ( 67